# Sample script for Hybrid VAE model

### imports

In [1]:
%matplotlib inline

from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../../Modules")
sys.path.append("../../Datasets")

# local imports
from visualize import printText
from models import LadderCNNVAE
from babelDatasets.sentiment140 import Sentiment140
from babelDatasets.utils import padding_merge

## Define dataset loader

In [2]:
batch_size = 32

dataset = Sentiment140(data_directory="../../Datasets/Data",train=False,max_sentence_size=32)
data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True,collate_fn=padding_merge)

num_classes = 82

batch_loader = iter(data_loader)
# input has shape [batch_size,seq_len,1]

# N sents: 52990  train: False  sentences_path: ../../Datasets/Data/test_sentences.txt


## Define and load model

In [12]:
model = LadderCNNVAE(input_size=1,hidden_sizes=[512,256,128],latent_sizes=[64,32,16],recon_hidden_size=256,output_size=num_classes,use_softmax=True)
model.eval()
model.load_state_dict(torch.load("../../Saved_models/Sentiment140/LCNNVAE_nll_map.pt"))

model_big = LadderCNNVAE(input_size=1,hidden_sizes=[512,256,128,64,32],latent_sizes=[128,64,32,16,8],recon_hidden_size=256,output_size=num_classes,use_softmax=True)
model_big.eval()
model_big.load_state_dict(torch.load("../../Saved_models/Sentiment140/LCNNVAE_nll_map_5l.pt"))

## Collect some samples and plot

Please note that this is not a reconstruction of "true" those are just to compare how some random samples from the dataset look like. And the samples below is new samples generated from the model

In [18]:
def sampleCompare(batch_loader,model,data_decoder):
    sampled_data = {"x":[],"recon_x":[]}
    try:
        batch = next(batch_loader)[0]
    except StopIteration:
        batch_loader = iter(data_loader)
        batch = next(batch_loader)[0]
    x = Variable(torch.FloatTensor(batch)).unsqueeze(2).transpose(1,0)
    size = (x.size()[0],x.size()[1],model.sample_size)
    z = Variable(torch.FloatTensor(*size).normal_(),requires_grad=False)
    recon_x = model.sample(z)
    _,topi = recon_x.data.topk(1)
    pred_x = Variable(topi)
    sampled_data["x"] = x.squeeze(2)
    sampled_data["recon_x"] = pred_x.squeeze(2)
    printText(sampled_data,data_decoder,recon_x_text="Sample")

In [19]:
sampleCompare(batch_loader,model,dataset.encoderDecoder)


True:
 is enjoying the weather.*******
Sample:
 is clla aociallg les h*********

True:
 @user you lucky sausage you****
Sample:
 Iach chiuloutank i dlopy fiiy**

True:
 Is going swimming again********
Sample:
 iast daran pi******************

True:
 i  the new dmb cd**************
Sample:
 ia up sadl Coon.***************

True:
 i love clean clothes***********
Sample:
 I'hEe are sroics****Itc********

True:
 changed my twitter name********
Sample:
 is meav somoks.. ailles evckt**

True:
 Oh yeah. I got a new camera na.
Sample:
 iasten. *ghy!!*****************

True:
 @user**************************
Sample:
 Iisvenr.***********************

True:
 @user Thats great!*************
Sample:
 i  musnygh!********************

True:
 sydney's coming over later.****
Sample:
 is boyso  oo****fe.****Ah's****

True:
 #iremember things**************
Sample:
 Iateellg spickes.is************


In [21]:
sampleCompare(batch_loader,model_big,dataset.encoderDecoder)


True:
 Soccer with the gang***********
Sample:
 Io  aray lad  m  *****  * re***

True:
 @user  thank you doll!*********
Sample:
 Iom celelnah leauncinan my.** *

True:
 Party tonight******************
Sample:
 Iotth aeotcing o ausely ****...

True:
 today was so relaxing**********
Sample:
 Iast ie oll i loenasn mone  * *

True:
 Today was a good sunday********
Sample:
 io  aen suttchfed.*** @un ar***

True:
 Goodmorning********************
Sample:
 @  aun yp. yeskin**************

True:
 livin the good life************
Sample:
 I  et wime  ara****************

True:
 In the recording studio!*******
Sample:
 I  ingy ail  Ilaiys io..   ate 

True:
 @user please follow me!!!!!****
Sample:
 @ mey re muaputing winugi e  . 

True:
 @userKillBoy nope I can see you
Sample:
 I st my s**********************

True:
 @user happy birthday***********
Sample:
 I  tred************************
